In [6]:
import numpy as np
import freud
import gsd.hoomd
import matplotlib.pyplot as plt

In [7]:
#Trim the initial file
scale=0.917
snap = gsd.hoomd.open('ideal_tiling.gsd', 'rb')[0]
box, positions  = [1000, 1000, 0, 0, 0, 0], snap.particles.position

aq = freud.locality.AABBQuery(box, positions)
nlist = aq.query(positions, {'r_max': 0.97, 'exclude_ii':True}).toNeighborList()
trimmed_positions = positions[nlist.neighbor_counts>0]

s = gsd.hoomd.Snapshot()
s.particles.N = len(trimmed_positions)
s.particles.types = ['A']
s.particles.typeid = ['0']*s.particles.N
s.particles.position = trimmed_positions

with gsd.hoomd.open(name='ideal_trimmed.gsd', mode='wb') as f:
    f.append(s)

In [8]:
#Write the neighbour and position lists
snap = gsd.hoomd.open('ideal_trimmed.gsd', 'rb')[0]
aq2 = freud.locality.AABBQuery(box, positions)
nlist2 = aq2.query(positions, {'r_max': 0.97, 'exclude_ii':True}).toNeighborList()
np.set_printoptions(suppress=True)
np.savetxt("P_ideal.txt", positions.flatten(), fmt='%4f')
np.savetxt("NL_ideal.txt", nlist2[:].flatten(), fmt='%4d')  

If necessary, now run the lifting code. Arguments following the executable's name are
-  The name of the gsd file to analyze
-  The number of queue threads
-  The number of neighbor list searching threads per queue thread
-  The number of visited list searching threads per neighbour list searching thread
-  The index of the origin point
-  The factor by which to scale the basis set

In [9]:
!./lift ideal 1 2 2 0 0.917

Time taken was 2.066519


In [10]:
#Test the projection
PI=np.pi
Q = np.array([[0, 1*scale, 0],                                             
              [np.sin(2*PI/5)*scale, np.cos(2*PI/5)*scale, 0],                        
              [np.sin(4*PI/5)*scale, np.cos(4*PI/5)*scale, 0],                        
              [np.sin(6*PI/5)*scale, np.cos(6*PI/5)*scale, 0],                        
              [np.sin(8*PI/5)*scale, np.cos(8*PI/5)*scale, 0]])

H = np.loadtxt('H_ideal.txt')
H = H.reshape((int(len(H)/5),5))

proj = Q.T @ H.T

#plt.scatter(proj[0], proj[1], s=0.01)
#plt.show()